In [58]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [59]:
train_paths = tf.io.matching_files('d:\\IT\\study\\study\\deeplearning\\final\\CIFAR100\\train\\*.png')
train_paths = train_paths.numpy()

test_paths = tf.io.matching_files('d:\\IT\\study\\study\\deeplearning\\final\\CIFAR100\\test\\*.png')
test_paths = test_paths.numpy()

In [60]:
classes = []
label_dict = dict()
label_dict_inv = dict()
for path in train_paths:
  path = path.decode()
  label = path[path.find('_')+1:path.find('.png')]
  
  if label not in label_dict:
    label_dict[label] = len(label_dict)

for key in label_dict:
  label_dict_inv[label_dict[key]] = key

print(label_dict)
print(label_dict_inv)

{'cattle': 0, 'motorcycle': 1, 'snail': 2, 'dolphin': 3, 'clock': 4, 'willow_tree': 5, 'hamster': 6, 'mushroom': 7, 'poppy': 8, 'tiger': 9, 'man': 10, 'orchid': 11, 'aquarium_fish': 12, 'kangaroo': 13, 'lamp': 14, 'butterfly': 15, 'boy': 16, 'fox': 17, 'squirrel': 18, 'rocket': 19, 'lizard': 20, 'pine_tree': 21, 'seal': 22, 'girl': 23, 'oak_tree': 24, 'road': 25, 'leopard': 26, 'bus': 27, 'otter': 28, 'bridge': 29, 'sea': 30, 'apple': 31, 'rabbit': 32, 'worm': 33, 'streetcar': 34, 'couch': 35, 'shrew': 36, 'tractor': 37, 'raccoon': 38, 'lion': 39, 'camel': 40, 'maple_tree': 41, 'snake': 42, 'mountain': 43, 'beaver': 44, 'caterpillar': 45, 'bowl': 46, 'whale': 47, 'table': 48, 'palm_tree': 49, 'plate': 50, 'beetle': 51, 'castle': 52, 'chair': 53, 'shark': 54, 'mouse': 55, 'train': 56, 'bed': 57, 'crab': 58, 'television': 59, 'porcupine': 60, 'wardrobe': 61, 'sweet_pepper': 62, 'bicycle': 63, 'bee': 64, 'bear': 65, 'pear': 66, 'spider': 67, 'flatfish': 68, 'elephant': 69, 'lawn_mower': 7

In [61]:
# train_x_path = []
# train_y = []
# for path in paths:
#   path = path.decode()
#   label = path[path.find('_')+1:path.find('.png')]
#   train_x_path.append(path)
#   train_y.append(label_dict[label])

In [62]:
def tf_read_image(path):
  gfile = tf.io.read_file(path)
  image = tf.io.decode_image(gfile, dtype=tf.float32)
  return image


def make_batch_x_y(batch_paths):
  batch_images = []
  batch_labels = []
  
  for path in batch_paths:
    path = path.decode()
    image = tf_read_image(path)
    label = label_dict[path[path.find('_')+1:path.find('.png')]]
    batch_images.append(image)
    batch_labels.append(label)
  return tf.convert_to_tensor(batch_images), tf.convert_to_tensor(batch_labels)

In [63]:
train_x_1d, train_y_1d = make_batch_x_y(train_paths)

In [ ]:
def Mymodel(input_shape, num_classes):
  my_inputs = tf.keras.layers.Input(shape=input_shape)   
  net = tf.keras.layers.Conv2D(32, 3, 1, padding="same", activation='relu')(my_inputs)           
  net = tf.keras.layers.Conv2D(32, 3, 1, padding="same", activation='relu')(net)             
  net = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(net)                                      
  net = tf.keras.layers.Dropout(0.2)(net)                                                    

  net = tf.keras.layers.Conv2D(64, 3, 1, padding="same", activation='relu')(net)            
  net = tf.keras.layers.Conv2D(64, 3, 1, padding="same", activation='relu')(net)            
  net = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(net)                                    
  net = tf.keras.layers.Dropout(0.2)(net)  

  net = tf.keras.layers.Flatten()(net)                                                      
  net = tf.keras.layers.Dense(512, activation='relu')(net)
  # net = tf.keras.layers.Dense(512)(net,  activation='relu')
  net = tf.keras.layers.Dropout(0.3)(net)
  net = tf.keras.layers.Dense(num_classes, activation="softmax")(net)                       
  model = tf.keras.Model(inputs=my_inputs, outputs=net, name="Final_Project")
  return model

In [ ]:
input_shape = (32, 32, 3)       # 높이, 폭, 채널수
num_classes = 100                # 100개로 설정
mymodel = Mymodel(input_shape, num_classes)
# mymodel.summary()
mymodel.compile(optimizer=tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
mymodel.summary()

In [ ]:
num_epochs = 5          
batch_size = 32
hist = mymodel.fit(train_x_1d, train_y_1d, batch_size = batch_size, shuffle = True, epochs = num_epochs)

In [ ]:
test_x_1d, test_y_1d = make_batch_x_y(test_paths)

results = mymodel.evaluate(test_x_1d, test_y_1d, batch_size=32)
print("test loss, test acc:", results)

In [ ]:
test_img = test_x_1d[6510]
output = mymodel.predict(test_img[tf.newaxis,...])
output = np.argmax(output)

print(label_dict_inv[output])
plt.imshow(test_img)